<a href="https://colab.research.google.com/github/ANSHIKA-26/Research-Fingerprint/blob/main/GAN_26_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install piq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os, gc, torch, numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
from tqdm import tqdm
from piq import ssim as piqi_ssim, psnr as piqi_psnr
from datetime import datetime


In [ ]:
import os

PROJECT_ROOT = "/content/drive/MyDrive/gan_2026"

DATA_ROOT = os.path.join(PROJECT_ROOT, "GAN2_LOCAL")
TRAIN_ROOT = os.path.join(DATA_ROOT, "train")
VAL_ROOT   = os.path.join(DATA_ROOT, "val")
TEST_ROOT  = os.path.join(DATA_ROOT, "test")

CHUNKS_ROOT = os.path.join(DATA_ROOT, "chunks_train")

CKPT_DIR = os.path.join(PROJECT_ROOT, "checkpoints")
LOG_DIR  = os.path.join(PROJECT_ROOT, "logs")
SAMPLE_DIR = os.path.join(PROJECT_ROOT, "samples")

CHECKPOINT_PATH = os.path.join(CKPT_DIR, "latest.pth")

os.makedirs(CHUNKS_ROOT, exist_ok=True)
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(SAMPLE_DIR, exist_ok=True)

print("✅ Paths ready")


✅ Paths ready


In [ ]:
import os
import gc
import torch
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
from tqdm import tqdm

from piq import ssim as piqi_ssim, psnr as piqi_psnr


In [ ]:
# ===== DEVICE =====
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# ===== TRAINING CONSTANTS =====
BATCH_SIZE = 32          # fixed as you wanted
NUM_CHUNKS = 4
EPOCHS_PER_CHUNK = 15
CYCLES = 2


Using device: cuda


In [ ]:
from PIL import Image, UnidentifiedImageError


In [ ]:
import os
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset
import torchvision.transforms as T

class PairedDataset(Dataset):
    def __init__(self, root, size=128):
        self.A_dir = os.path.join(root, "A")
        self.B_dir = os.path.join(root, "B")

        # keep only paired filenames
        self.names = sorted([
            f for f in os.listdir(self.A_dir)
            if os.path.exists(os.path.join(self.B_dir, f))
        ])

        self.transform = T.Compose([
            T.Grayscale(num_output_channels=1),  # 🔹 force 1 channel
            T.Resize((size, size)),
            T.ToTensor(),
            T.Normalize((0.5,), (0.5,))          # 🔹 correct for grayscale
        ])

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        A_path = os.path.join(self.A_dir, name)
        B_path = os.path.join(self.B_dir, name)

        try:
            A = Image.open(A_path).convert("RGB")
            B = Image.open(B_path).convert("RGB")
        except (UnidentifiedImageError, OSError):
            print(f"⚠️ Skipping corrupted image: {name}")
            return self.__getitem__((idx + 1) % len(self))

        return self.transform(A), self.transform(B), name


In [ ]:
# quick test on one chunk
test_dataset = PairedDataset(
    os.path.join(CHUNKS_ROOT, "chunk_0")
)

print("Total samples in chunk_0:", len(test_dataset))

A, B, name = test_dataset[0]
print("Latent shape:", A.shape)
print("GT shape:", B.shape)
print("Filename:", name)


Total samples in chunk_0: 4900
Latent shape: torch.Size([1, 128, 128])
GT shape: torch.Size([1, 128, 128])
Filename: 2302_11010.png


In [ ]:
# ===== adaptive batch size (GPU safe + CPU safe) =====
if DEVICE.type == "cpu":
    EFFECTIVE_BATCH_SIZE = min(16, BATCH_SIZE)
else:
    EFFECTIVE_BATCH_SIZE = BATCH_SIZE

print("Effective batch size:", EFFECTIVE_BATCH_SIZE)


Effective batch size: 32


In [ ]:
# ===== Train DataLoader (example: chunk_0) =====
train_dataset = PairedDataset(
    os.path.join(CHUNKS_ROOT, "chunk_0")
)

train_loader = DataLoader(
    train_dataset,
    batch_size=EFFECTIVE_BATCH_SIZE,
    shuffle=True,        # important for GAN training
    num_workers=0        # safest on Colab (CPU & GPU)
)


In [ ]:
# ===== Validation DataLoader (full validation set) =====
val_dataset = PairedDataset(VAL_ROOT)

val_loader = DataLoader(
    val_dataset,
    batch_size=EFFECTIVE_BATCH_SIZE,
    shuffle=False,       # NEVER shuffle validation
    num_workers=0
)


In [ ]:
# one batch sanity check
A, B, names = next(iter(train_loader))

print("Train batch shape A:", A.shape)
print("Train batch shape B:", B.shape)
print("First filename:", names[0])


Train batch shape A: torch.Size([32, 1, 128, 128])
Train batch shape B: torch.Size([32, 1, 128, 128])
First filename: 2484_6269.png


In [ ]:
# ===============================
# MODEL ARCHITECTURE (UNCHANGED)
# ===============================

class ResBlock(nn.Module):
    def __init__(self, i, o, stride=1):
        super().__init__()
        self.c1 = nn.Conv2d(i, o, 3, stride=stride, padding=1)
        self.n1 = nn.InstanceNorm2d(o)
        self.c2 = nn.Conv2d(o, o, 3, padding=1)
        self.n2 = nn.InstanceNorm2d(o)
        self.skip = nn.Conv2d(i, o, 1, stride=stride) if i != o or stride != 1 else nn.Identity()
        self.relu = nn.ReLU(True)

    def forward(self, x):
        r = self.skip(x)
        x = self.relu(self.n1(self.c1(x)))
        x = self.n2(self.c2(x))
        return self.relu(x + r)


class Down(nn.Module):
    def __init__(self, i, o):
        super().__init__()
        self.block = ResBlock(i, o, stride=2)

    def forward(self, x):
        s = self.block(x)
        return s, s


class Up(nn.Module):
    def __init__(self, i, skip, o):
        super().__init__()
        self.up = nn.ConvTranspose2d(i, o, 2, stride=2)
        self.block = ResBlock(o + skip, o)

    def forward(self, x, s):
        x = self.up(x)
        if x.shape[-1] != s.shape[-1]:
            s = F.interpolate(s, (x.shape[-2], x.shape[-1]))
        return self.block(torch.cat([x, s], 1))


class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        base = 64
        self.in_conv = nn.Conv2d(1, base, 1)

        self.d0 = Down(base, base * 2)
        self.d1 = Down(base * 2, base * 4)
        self.d2 = Down(base * 4, base * 8)
        self.d3 = Down(base * 8, base * 16)

        self.bottle = ResBlock(base * 16, base * 32)

        self.u3 = Up(base * 32, base * 16, base * 16)
        self.u2 = Up(base * 16, base * 8, base * 8)
        self.u1 = Up(base * 8, base * 4, base * 4)
        self.u0 = Up(base * 4, base * 2, base * 2)

        self.out = nn.Conv2d(base * 2, 1, 1)

    def forward(self, x):
        x, s0 = self.d0(self.in_conv(x))
        x, s1 = self.d1(x)
        x, s2 = self.d2(x)
        x, s3 = self.d3(x)

        x = self.bottle(x)

        x = self.u3(x, s3)
        x = self.u2(x, s2)
        x = self.u1(x, s1)
        x = self.u0(x, s0)

        return self.out(x)


class MultiScaleD(nn.Module):
    def __init__(self):
        super().__init__()
        base = 64
        layers = []

        for i, o in [(2, base), (base, base * 2), (base * 2, base * 4), (base * 4, base * 8)]:
            layers += [
                nn.Conv2d(i, o, 4, 2, 1),
                nn.InstanceNorm2d(o),
                nn.LeakyReLU(0.2)
            ]

        self.trunk = nn.Sequential(*layers)
        self.h1 = nn.Conv2d(base * 8, 1, 1)
        self.h2 = nn.Conv2d(base * 8, 1, 1)
        self.h3 = nn.Conv2d(base * 8, 1, 1)

    def forward(self, x):
        f = self.trunk(x)
        return [
            self.h1(f),
            self.h2(F.avg_pool2d(f, 2)),
            self.h3(F.avg_pool2d(f, 4))
        ]


# ===============================
# LOSS FUNCTION (UNCHANGED)
# ===============================

from piq import LPIPS
LPIPS_MODEL = LPIPS().to(DEVICE).eval()

class GANLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.L1Loss()

    def forward(self, real, fake, dFake, dReal):
        lp = LPIPS_MODEL(fake, real).mean()

        adv = (
            sum(F.mse_loss(h, torch.zeros_like(h)) for h in dFake) +
            sum(F.mse_loss(h, torch.ones_like(h)) for h in dReal)
        )

        return adv + 10 * self.l1(fake, real) + 0.05 * lp, {
            "Adv": adv.item(),
            "LPIPS": lp.item()
        }


Downloading: "https://github.com/photosynthesis-team/photosynthesis.metrics/releases/download/v0.4.0/lpips_weights.pt" to /root/.cache/torch/hub/checkpoints/lpips_weights.pt


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
print(Generator)
print(MultiScaleD)
print(GANLoss)


<class '__main__.Generator'>
<class '__main__.MultiScaleD'>
<class '__main__.GANLoss'>


In [ ]:
# ===== Instantiate models =====
G = Generator().to(DEVICE)
D = MultiScaleD().to(DEVICE)

print("Generator and Discriminator created.")


Generator and Discriminator created.


In [ ]:
# ===== optimizers =====
optG = optim.Adam(
    G.parameters(),
    lr=2e-5,
    betas=(0.5, 0.999)
)

optD = optim.Adam(
    D.parameters(),
    lr=2e-4,
    betas=(0.5, 0.999)
)

print("✅ Optimizers created")


✅ Optimizers created


In [ ]:
# ===== loss function =====
loss_fn = GANLoss().to(DEVICE)
print("✅ Loss function ready")


✅ Loss function ready


In [ ]:
print("G device:", next(G.parameters()).device)
print("D device:", next(D.parameters()).device)

g_params = sum(p.numel() for p in G.parameters())
d_params = sum(p.numel() for p in D.parameters())

print(f"Generator parameters: {g_params:,}")
print(f"Discriminator parameters: {d_params:,}")


G device: cuda:0
D device: cuda:0
Generator parameters: 129,764,737
Discriminator parameters: 2,757,059


In [ ]:
start_cycle = 0
start_chunk = 0
start_epoch = 0

if os.path.exists(CHECKPOINT_PATH):
    ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

    # restore weights
    G.load_state_dict(ckpt["G"])
    D.load_state_dict(ckpt["D"])
    optG.load_state_dict(ckpt["optG"])
    optD.load_state_dict(ckpt["optD"])

    # restore training position
    start_cycle = ckpt["cycle"]
    start_chunk = ckpt["chunk"]
    start_epoch = ckpt["epoch"] + 1  # resume from NEXT epoch

    # ---- if epoch finished, move to next chunk ----
    if start_epoch >= EPOCHS_PER_CHUNK:
        start_epoch = 0
        start_chunk += 1

    # ---- if chunk finished, move to next cycle ----
    if start_chunk >= NUM_CHUNKS:
        start_chunk = 0
        start_cycle += 1

    print(
        "🔄 RESUME MODE\n"
        f"Continuing from → Cycle {start_cycle}, "
        f"Chunk {start_chunk}, Epoch {start_epoch}"
    )
else:
    print("🆕 No checkpoint found → Starting fresh training")


🔄 RESUME MODE
Continuing from → Cycle 1, Chunk 0, Epoch 6


In [ ]:
import random

def epoch_median_ssim(fake, real, k=5):
    """
    fake, real: [B, 1, H, W] in range [-1, 1]
    returns median SSIM over k random samples
    """
    b = fake.size(0)
    k = min(k, b)

    idx = random.sample(range(b), k)

    fake_sel = fake[idx]
    real_sel = real[idx]

    # convert [-1,1] → [0,1] and CLAMP
    fake_sel = ((fake_sel + 1) / 2).clamp(0.0, 1.0)
    real_sel = ((real_sel + 1) / 2).clamp(0.0, 1.0)

    ssim_vals = piqi_ssim(fake_sel, real_sel, reduction='none')
    return torch.median(ssim_vals).item()


In [ ]:
for cycle in range(start_cycle, CYCLES):

    for chunk in range(NUM_CHUNKS):

        if cycle == start_cycle and chunk < start_chunk:
            continue

        print(f"\n📦 Cycle {cycle}, Chunk {chunk}")

        train_dataset = PairedDataset(
            os.path.join(CHUNKS_ROOT, f"chunk_{chunk}")
        )

        train_loader = DataLoader(
            train_dataset,
            batch_size=EFFECTIVE_BATCH_SIZE,
            shuffle=True,
            num_workers=0
        )

        for epoch in range(EPOCHS_PER_CHUNK):

            if (
                cycle == start_cycle and
                chunk == start_chunk and
                epoch < start_epoch
            ):
                continue

            print(f"\n🔁 Epoch {epoch}")

            G.train()
            D.train()

            epoch_ssim_vals = []   # ← collect SSIMs for this epoch

            for A, B, _ in tqdm(train_loader):
                A = A.to(DEVICE)
                B = B.to(DEVICE)

                # -------- Train Discriminator --------
                optD.zero_grad()

                fake = G(A).detach()

                lossD = 0.5 * (
                    sum(F.mse_loss(h, torch.ones_like(h))
                        for h in D(torch.cat([A, B], 1))) +
                    sum(F.mse_loss(h, torch.zeros_like(h))
                        for h in D(torch.cat([A, fake], 1)))
                )

                lossD.backward()
                optD.step()

                # -------- Train Generator --------
                optG.zero_grad()

                fake = G(A)
                lossG, _ = loss_fn(
                    B,
                    fake,
                    D(torch.cat([A, fake], 1)),
                    D(torch.cat([A, B], 1))
                )

                lossG.backward()
                optG.step()

                # ---- epoch-level SSIM (5 random samples) ----
                with torch.no_grad():
                    ssim_val = epoch_median_ssim(fake, B, k=5)
                    epoch_ssim_vals.append(ssim_val)

            # ---- epoch summary ----
            epoch_ssim = np.median(epoch_ssim_vals)
            print(
                f"📊 Epoch {epoch} summary | "
                f"Median SSIM (5-sample) = {epoch_ssim:.4f}"
            )

            # -------- SAVE CHECKPOINT --------
            torch.save({
                "cycle": cycle,
                "chunk": chunk,
                "epoch": epoch,
                "G": G.state_dict(),
                "D": D.state_dict(),
                "optG": optG.state_dict(),
                "optD": optD.state_dict()
            }, CHECKPOINT_PATH)

            print(
                f"💾 Overwriting latest.pth "
                f"(cycle={cycle}, chunk={chunk}, epoch={epoch})"
            )

        # -------- VALIDATION (whole set, MEDIAN) --------
        print("🔍 Running validation...")

        G.eval()
        ssim_vals, psnr_vals = [], []

        with torch.no_grad():
            for A, B, _ in val_loader:
                A = A.to(DEVICE)
                B = B.to(DEVICE)

                fake = ((G(A) + 1) / 2).clamp(0.0, 1.0)
                real = ((B + 1) / 2).clamp(0.0, 1.0)

                ssim_batch = piqi_ssim(fake, real, reduction='none')
                psnr_batch = piqi_psnr(fake, real, reduction='none')

                ssim_vals.extend(ssim_batch.cpu().numpy())
                psnr_vals.extend(psnr_batch.cpu().numpy())

        print(
            f"✅ Validation result | "
            f"Median SSIM = {np.median(ssim_vals):.4f}, "
            f"Median PSNR = {np.median(psnr_vals):.2f}"
        )

        gc.collect()



📦 Cycle 1, Chunk 0

🔁 Epoch 6


 53%|█████▎    | 81/154 [14:19<11:33,  9.50s/it]

⚠️ Skipping corrupted image: 2415_18790.png


100%|██████████| 154/154 [24:53<00:00,  9.70s/it]


📊 Epoch 6 summary | Median SSIM (5-sample) = 0.6682
💾 Overwriting latest.pth (cycle=1, chunk=0, epoch=6)

🔁 Epoch 7


...

 66%|██████▌   | 102/154 [04:37<02:02,  2.35s/it]

⚠️ Skipping corrupted image: 2415_18790.png


100%|██████████| 154/154 [06:39<00:00,  2.60s/it]


📊 Epoch 13 summary | Median SSIM (5-sample) = 0.6771
💾 Overwriting latest.pth (cycle=1, chunk=0, epoch=13)

🔁 Epoch 14


  3%|▎         | 5/154 [00:12<05:54,  2.38s/it]